# Prepping JPL and OTIS data for ingestion to the JWST Calibration Pipeline: Overview and Initialization

This notebook generates JWST cal pipeline-ready FITS files - filename is ([original filename - '.fits'] + '_pipe.fits') -  from JPL LVL1 data (with standard subarrays/FULL array) and OTIS LVL1 data. 

For JPL data, I use Generate_JPL_Pipeline_Ready_File function to generate a pipeline-ready FITS file from LVL1 JPL data. It has been tested on JPL6, JPL7, and JPL8 files - the JWST Detector1Pipeline can then be run on these '*_pipe.fits' files. It currently does not work for JPL data with non-standard subarrays.

Notes on the Generate_JPL_Pipeline_Ready_File function:
     - 'DETECTOR' keyword hard-coded to 'MIRIMAGE'
     - 'READPATT' keyword hard-coded to 'FAST'
     - 'GROUPGAP' keyword hard-coded to 0 ### is this always 0 for MIRI? GROUPGAP is "The number of dropped frames in between groups."
     - 'SCA_ID'   dependent on hard-coded value for 'DETECTOR' - 'MIRIMAGE' gives SCA_ID =  493
     
The JPL data incorrectly uses populates the 'COLSTART' keyword - see http://poppy.as.arizona.edu/dhas/ for more details: "...it was determined that for JPL testing the COLSTART keyword in the header is incorrect. This version of the DHAS fixes the COLSTART value in the software. It does not update the COLSTART in the RAW DATA." 

To get the correct 'COLSTART' value, we do: hdr['COLSTART'] = int(0.2*hdr['COLSTART'] + 0.8) <--- this correction now matches the COLSTART keywords found in all the OTIS data. The JWST pipeline uses 'SUBSTRT1' and 'SUBSTRT2' keywords to get the starting lower-left corner of the subarray. With the correction to 'COLSTART', hdr['SUBSTRT1'] = (hdr['COLSTART'] * 4 - 3) and hdr['SUBSTRT2'] = hdr['ROWSTART'] (noting that 'ROWSTART' is a keyword found in the JPL data).

In [ ]:
from pipefits import Generate_JPL_Pipeline_Ready_File, Generate_OTIS_Pipeline_Ready_File
import os.path
from astropy.io import fits

In [ ]:
### place jpl/otis FITS files in same directory as this notebook
local_data_directory = os.getcwd() + '/'
jpl_file = 'MIRI_5582_211_S_20180308-035345_SCE1.fits' ### found in /ifs/jwst/wit/witserv/data19/miri/JPL8/11_All_Subarray/
otis_file = 'MIRDARK-BRIGHTSKY-7242235622_1_493_SE_2017-08-31T00h06m36.fits' ### found in /ifs/jwst/wit/witserv/data19/miri/OTIS/ 

# JPL

## Generate JPL file (*_pipe.fits) that can feed into the JWST pipeline

In [ ]:
data_file_path_jpl = local_data_directory + jpl_file
original_jpl_file = fits.open(data_file_path_jpl)
### show info for the original file
original_jpl_file.info()
original_jpl_file.close()

In [ ]:
Generate_JPL_Pipeline_Ready_File(data_file_path_jpl)

In [ ]:
### show info for newly generated *_pipe.fits file
jpl_pipeline_ready_filepath = data_file_path_jpl.replace(".fits","_pipe.fits")
raw_ramp_hdu = fits.open(jpl_pipeline_ready_filepath)
raw_ramp_hdu.info()
raw_ramp_hdu.close()

## Run JWST Pipeline on JPL "_pipe.fits" File

In [ ]:
from jwst.pipeline import Detector1Pipeline

In [ ]:
""" This cell runs the JWST Detector1Pipeline on the JPL "*_pipe.fits" file. I only care about generating the corrected ramp file (*_ramp.fits) file,
 so I skip the ramp_fit and gain_scale steps. The save_calibrated_ramp and save_results options are necessary for writing the corrected ramp file"""
mypipeline = Detector1Pipeline()
mypipeline.ramp_fit.skip = True
mypipeline.gain_scale.skip = True
mypipeline.save_calibrated_ramp = True
mypipeline.save_results = True
result = mypipeline.run(jpl_pipeline_ready_filepath)

In [ ]:
corrected_ramp_dir = jpl_pipeline_ready_filepath.replace(".fits","_ramp.fits")
corrected_ramp_hdu = fits.open(corrected_ramp_dir)
corrected_ramp_hdu.info()

# OTIS

## Generate OTIS file (*_pipe.fits) that can feed into the JWST pipeline

In [ ]:
data_file_path_otis = local_data_directory + otis_file

In [ ]:
Generate_OTIS_Pipeline_Ready_File(data_file_path_otis)

In [ ]:
original_otis_file = fits.open(data_file_path_otis)
original_otis_file.info()
original_otis_file.close()

In [ ]:
otis_pipeline_ready_filepath = data_file_path_otis.replace(".fits","_pipe.fits")
raw_ramp_hdu = fits.open(otis_pipeline_ready_filepath)
raw_ramp_hdu.info()
raw_ramp_hdu.close()

## Run JWST Pipeline on OTIS "_pipe.fits" File

In [ ]:
from jwst.pipeline import Detector1Pipeline

In [ ]:
""" This cell runs the JWST Detector1Pipeline on the OTIS "*_pipe.fits" file. I only care about generating the corrected ramp file (*_ramp.fits) file,
 so I skip the ramp_fit and gain_scale steps. The save_calibrated_ramp and save_results options are necessary for writing the corrected ramp file"""
mypipeline = Detector1Pipeline()
mypipeline.ramp_fit.skip = True
mypipeline.gain_scale.skip = True
mypipeline.save_calibrated_ramp = True
mypipeline.save_results = True
result = mypipeline.run(otis_pipeline_ready_filepath)

In [ ]:
corrected_ramp_dir = otis_pipeline_ready_filepath.replace(".fits","_ramp.fits")
corrected_ramp_hdu = fits.open(corrected_ramp_dir)
corrected_ramp_hdu.info()

# Future Work for non-standard JPL subarray files

The function "Generate_JPL_Pipeline_Ready_File" will work on a JPL FITS file with a non-standard subarray. However when I try to run the JWST pipeline on a file like this, it crashes as soon as it tries to find an appropriate reference file. The user can override the default reference files and specify there own reference files to use for each pipeline step.

In [ ]:
''' This JPL FITS file contains data for a non-standard subarray - it mimics the SUB64 subarray but is slightly smaller 
in size and placed in a different location on the FULL array. With reference pixels, SUB64 should be of size [72, 80], not [64,80]'''

jpl_file = 'MIRI_5450_1_S_20160908-005910_SCE1.fits' ### found in /ifs/jwst/wit/witserv/data19/miri/JPL6/A_Exoplanets/
data_file_path_jpl = local_data_directory + jpl_file
original_jpl_file = fits.open(data_file_path_jpl)
### show info for the original file
original_jpl_file.info()
original_jpl_file.close()

In [ ]:
### the Generate_JPL_Pipeline_Ready_File successfully runs on this file
Generate_JPL_Pipeline_Ready_File(data_file_path_jpl)

In [ ]:
### show info for newly generated *_pipe.fits file
jpl_pipeline_ready_filepath = data_file_path_jpl.replace(".fits","_pipe.fits")
raw_ramp_hdu = fits.open(jpl_pipeline_ready_filepath)
raw_ramp_hdu.info()
raw_ramp_hdu.close()

In [ ]:
from jwst.pipeline import Detector1Pipeline

In [ ]:
### create instance of JWST pipeline
mypipeline = Detector1Pipeline()
mypipeline.ramp_fit.skip = True
mypipeline.gain_scale.skip = True
mypipeline.save_calibrated_ramp = True
mypipeline.save_results = True

In [ ]:
### print reference file types
print(mypipeline.reference_file_types)
### print pipeline steps
mypipeline.step_defs

In [ ]:
### lines to override the reference files used for different steps of the Detector1Pipeline - TODO: need to change these to appropriate reference files
mypipeline.dark_current.override_dark = '/grp/crds/cache/references/jwst/jwst_miri_dark_0058.fits'
mypipeline.gain_scale.override_gain = '/grp/crds/cache/references/jwst/jwst_miri_gain_0008.fits'
mypipeline.ipc.override_ipc = '/grp/crds/cache/references/jwst/jwst_miri_ipc_0009.fits'
mypipeline.linearity.override_linearity = '/grp/crds/cache/references/jwst/jwst_miri_linearity_0024.fits'
# mypipeline.rscd.override_rscd = TBD

In [ ]:
### running the pipeline
result = mypipeline.run(jpl_pipeline_ready_filepath)